In [1]:
import os, time, itertools, imageio, pickle
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/matplotlib/__init__.py:962: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #2
  (fname, cnt))
/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/matplotlib/__init__.py:962: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #3
  (fname, cnt))
/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [27]:
def generator(x):
    # initializers
    w_init = tf.truncated_normal_initializer(mean=0, stddev=0.02)
    b_init = tf.constant_initializer(0.)
    # 1st hidden layer
    w0 = tf.get_variable('G_w0', [x.get_shape()[1], 128], initializer=w_init)
    b0 = tf.get_variable('G_b0', [128], initializer=b_init)
    h0 = tf.nn.relu(tf.matmul(x, w0) + b0)
    # output hidden layer
    w1 = tf.get_variable('G_w1', [h0.get_shape()[1], 784], initializer=w_init)
    b1 = tf.get_variable('G_b1', [784], initializer=b_init)
    o = tf.nn.tanh(tf.matmul(h0, w1) + b1)
    return o
    ### Code:ToDO( Change the architecture as CW2 Guidance required)

In [28]:
def discriminator(x, drop_out):
    # initializers
    w_init = tf.truncated_normal_initializer(mean=0, stddev=0.02)
    b_init = tf.constant_initializer(0.)
    # 1st hidden layer
    w0 = tf.get_variable('D_w0', [x.get_shape()[1], 784], initializer=w_init)
    b0 = tf.get_variable('D_b0', [784], initializer=b_init)
    h0 = tf.nn.relu(tf.matmul(x, w0) + b0)
    # output layer
    w1 = tf.get_variable('D_w1', [h0.get_shape()[1], 1], initializer=w_init)
    b1 = tf.get_variable('D_b1', [1], initializer=b_init)
    o = tf.sigmoid(tf.matmul(h0, w1) + b1)
    ###  Code: ToDO( Change the architecture as CW2 Guidance required)

    return o

In [29]:
def show_result(num_epoch, show = False, save = False, path = 'result.png'):
    z_ = np.random.normal(0, 1, (25, 100))    # z_ is the input of generator, every epochs will random produce input
    ##Code:ToDo complete the rest of part

In [30]:
def show_train_hist(hist, show = False, save = False, path = 'Train_hist.png'):
    x = range(len(hist['D_losses']))

    y1 = hist['D_losses']
    y2 = hist['G_losses']

    plt.plot(x, y1, label='D_loss')
    plt.plot(x, y2, label='G_loss')

    plt.xlabel('Epoch')
    plt.ylabel('Loss')

    plt.legend(loc=4)
    plt.grid(True)
    plt.tight_layout()

    if save:
        plt.savefig(path)

    if show:
        plt.show()
    else:
        plt.close()

In [31]:
# training parameters
batch_size = 256
lr = 0.0002
train_epoch = 200

# load MNIST
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)
train_set = (mnist.train.images - 0.5) / 0.5  # normalization; range: -1 ~ 1

# networks : generator
with tf.variable_scope('G'):
    z = tf.placeholder(tf.float32, shape=(None, 100))
    G_z = generator(z)
# networks : discriminator
with tf.variable_scope('D') as scope:
    drop_out = tf.placeholder(dtype=tf.float32, name='drop_out')
    x = tf.placeholder(tf.float32, shape=(None, 784))
    D_real = discriminator(x, drop_out)
    scope.reuse_variables()
    D_fake = discriminator(G_z, drop_out)

# loss for each network
eps = 1e-2
D_loss = tf.reduce_mean(-tf.log(D_real + eps) - tf.log(1 - D_fake + eps))
G_loss = tf.reduce_mean(-tf.log(D_fake + eps))

# trainable variables for each network
t_vars = tf.trainable_variables()
D_vars = [var for var in t_vars if 'D_' in var.name]
G_vars = [var for var in t_vars if 'G_' in var.name]

# optimizer for each network
D_optim = tf.train.AdamOptimizer(lr).minimize(D_loss, var_list=D_vars)
G_optim = tf.train.AdamOptimizer(lr).minimize(G_loss, var_list=G_vars)

# open session and initialize all variables
sess = tf.InteractiveSession()
tf.global_variables_initializer().run()

# results save folder
if not os.path.isdir('MNIST_GAN_results'):
    os.mkdir('MNIST_GAN_results')
if not os.path.isdir('MNIST_GAN_results/results'):
    os.mkdir('MNIST_GAN_results/results')
train_hist = {}
train_hist['D_losses'] = []
train_hist['G_losses'] = []
train_hist['per_epoch_ptimes'] = []
train_hist['total_ptime'] = []
# training-loop
np.random.seed(int(time.time()))
start_time = time.time()
for epoch in range(train_epoch):
    G_losses = []
    D_losses = []
    epoch_start_time = time.time()
    for iter in range(train_set.shape[0] // batch_size):
        # update discriminator
        x_ = train_set[iter*batch_size:(iter+1)*batch_size]
        z_ = np.random.normal(0, 1, (batch_size, 100))

        loss_d_, _ = sess.run([D_loss, D_optim], {x: x_, z: z_, drop_out: 0.3})
        D_losses.append(loss_d_)

        # update generator
        z_ = np.random.normal(0, 1, (batch_size, 100))
        loss_g_, _ = sess.run([G_loss, G_optim], {z: z_, drop_out: 0.3})
        G_losses.append(loss_g_)

    epoch_end_time = time.time()
    per_epoch_ptime = epoch_end_time - epoch_start_time
    print('[%d/%d] - ptime: %.2f loss_d: %.3f, loss_g: %.3f' % ((epoch + 1), train_epoch, per_epoch_ptime, np.mean(D_losses), np.mean(G_losses)))

    ### Code: TODO Code complet show_result function)
    ###p = 'MNIST_GAN_results/results/MNIST_GAN_' + str(epoch + 1) + '.png'
    ###show_result((epoch + 1), save=True, path=p)
    train_hist['D_losses'].append(np.mean(D_losses))
    train_hist['G_losses'].append(np.mean(G_losses))
    train_hist['per_epoch_ptimes'].append(per_epoch_ptime)
end_time = time.time()
total_ptime = end_time - start_time
train_hist['total_ptime'].append(total_ptime)
print('Avg per epoch ptime: %.2f, total %d epochs ptime: %.2f' % (np.mean(train_hist['per_epoch_ptimes']), train_epoch, total_ptime))
print("Training finish!... save training results")
with open('MNIST_GAN_results/train_hist.pkl', 'wb') as f:
    pickle.dump(train_hist, f)
show_train_hist(train_hist, save=True, path='MNIST_GAN_results/MNIST_GAN_train_hist.png')
images = []
sess.close()

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
[1/200] - ptime: 2.42 loss_d: 0.665, loss_g: 0.880
[2/200] - ptime: 2.39 loss_d: 0.379, loss_g: 1.150
[3/200] - ptime: 2.35 loss_d: 0.241, loss_g: 1.521
[4/200] - ptime: 2.39 loss_d: 0.122, loss_g: 2.415
[5/200] - ptime: 2.36 loss_d: 0.165, loss_g: 2.452
[6/200] - ptime: 2.35 loss_d: 0.153, loss_g: 2.926
[7/200] - ptime: 2.35 loss_d: 0.256, loss_g: 2.535
[8/200] - ptime: 2.36 loss_d: 0.268, loss_g: 2.590
[9/200] - ptime: 2.34 loss_d: 0.209, loss_g: 2.855
[10/200] - ptime: 2.40 loss_d: 0.085, loss_g: 3.710
[11/200] - ptime: 2.46 loss_d: 0.133, loss_g: 3.137
[12/200] - ptime: 2.34 loss_d: 0.182, loss_g: 2.699
[13/200] - ptime: 2.35 loss_d: 0.128, loss_g: 2.994
[14/200] - ptime: 2.37 loss_d: 0.074, loss_g: 3.540
[15/200] - ptime: 2.35 loss_d: 0.220, loss_g: 2.881
[16/200] - ptime: 2.35 loss_d: 0.26

[154/200] - ptime: 2.35 loss_d: 0.851, loss_g: 2.565
[155/200] - ptime: 2.36 loss_d: 0.941, loss_g: 2.417
[156/200] - ptime: 2.34 loss_d: 1.027, loss_g: 2.336
[157/200] - ptime: 2.34 loss_d: 0.932, loss_g: 2.486
[158/200] - ptime: 2.34 loss_d: 0.920, loss_g: 2.407
[159/200] - ptime: 2.36 loss_d: 0.923, loss_g: 2.486
[160/200] - ptime: 2.39 loss_d: 0.942, loss_g: 2.421
[161/200] - ptime: 2.38 loss_d: 0.839, loss_g: 2.482
[162/200] - ptime: 2.37 loss_d: 0.915, loss_g: 2.434
[163/200] - ptime: 2.35 loss_d: 0.766, loss_g: 2.624
[164/200] - ptime: 2.34 loss_d: 0.788, loss_g: 2.659
[165/200] - ptime: 2.36 loss_d: 0.735, loss_g: 2.665
[166/200] - ptime: 2.34 loss_d: 0.804, loss_g: 2.666
[167/200] - ptime: 2.35 loss_d: 0.629, loss_g: 2.971
[168/200] - ptime: 2.35 loss_d: 0.573, loss_g: 2.990
[169/200] - ptime: 2.34 loss_d: 0.520, loss_g: 3.080
[170/200] - ptime: 2.34 loss_d: 0.471, loss_g: 3.132
[171/200] - ptime: 2.36 loss_d: 0.586, loss_g: 3.018
[172/200] - ptime: 2.44 loss_d: 0.565, loss_g:

In [26]:
tf.reset_default_graph()